In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
np.random.seed(1234)

from simulate_module import *

In [2]:
sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}

In [3]:
args = {
    "n_tasks": 15,
    "conservative": True,
    "target_test_size": 0.8,
    "model_type": "lm",
    "base_output_dir": "derived_data"
}

In [4]:
if args["model_type"] == "lm":
    model_class = "lm"
    loss_fn = mse
elif args["model_type"] == "nn":
    model_class = "nn"
    loss_fn =  torch.nn.MSELoss()

In [24]:
for s in sigma_setting:
    # set directory
    if args["conservative"]:
        data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/conservative_derived_data")
    else:
        data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/derived_data")
    data_path = Path(data_path)
    working_path = data_path / s
    working_path.mkdir(parents = True, exist_ok = True)
    # generate data ------------------------------------------------
    np.random.seed(1234)
    f, betas, zs = random_functions(args["n_tasks"], 6,
                                    sigma_between = sigma_setting[s][0],
                                    sigma_within = sigma_setting[s][-1])
    result = []
    for i, fi in enumerate(f):
        x = np.random.uniform(0, 1, 100)
        result.append({
            "task": i,
            "x": x,
            "f": fi(x),
            "y": fi(x) + np.random.normal(0, .1, len(x))
        })
    # save data
    data_df = pd.concat([pd.DataFrame(r) for r in result])
    data_df.to_csv(working_path / "tasks.csv", index = False)
    data_df = data_df.reset_index()
    betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
    betas_df = pd.DataFrame(betas_df)
    betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
    betas_df.to_csv(working_path / "betas.csv", index = False)
    data_dict = data_df.to_dict(orient = "list")
    # bandit selection ------------------------------------------------
    for target_task in range(args["n_tasks"]):
        # prepare input
        input_data = prepare_input(data_dict,
                                   target_task = target_task,
                                   target_test_size = args["target_test_size"],
                                  preprocess = True)
        pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                               index = False)
        # bandit selection
        losses, alpha, beta, bandit_selects, pi, bl = bandit_source_train(input_data = input_data,
                                                                          model = model_class,
                                                                          batch_size = 8,
                                                                          decay_rate = .5,
                                                                          n_it = 100,
                                                                          loss_fn =  loss_fn,
                                                                          conservative = args["conservative"])
        # save outputs
        output_dir = working_path / f"target_{target_task}_{args['target_test_size']}"
        save_files(output_dir, alpha, beta, losses, bandit_selects, pi, bl)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 1)

try one script

In [3]:
def basis(x, knots=None, degree = 10):
    if knots is None:
        knots = np.linspace(0, 1, 10)
        
    H = [np.ones(len(x))]
    for k in range(len(knots)):
        for j in range(1, degree + 1):
            H.append(pos(x - knots[k]) ** j)
    
    return np.vstack(H).T
def random_functions(n_tasks, k_clusters, sigma_between, sigma_within):
    np.random.seed(1234)
    betas, zs = gaussian_mixture(n_tasks, k_clusters, sigma_between, sigma_within)
    functions = []
    for beta in betas:
        functions.append(spline_fun(beta))
        
    return functions, betas, zs

def gaussian_mixture(n_samples, k_clusters, sigma_between, sigma_within, dim = 101):
    means = np.random.normal(0, sigma_between, (k_clusters, dim))
    cluster_props = np.random.dirichlet(k_clusters * [1.5])
    
    betas, zs = [], []
    for task in range(n_samples):
        z = np.random.choice(range(k_clusters), p = cluster_props)
        delta = np.random.normal(0, sigma_within, (1, dim))
        betas.append(means[z] + delta)
        zs.append(z)
        
    return np.vstack(betas), zs
        
def spline_fun(beta):
    def f(x):
        return np.matmul(basis(x), beta)
    return f
    



In [4]:
base_dir = Path("degree10_scale_epochs50")
base_dir.mkdir(exist_ok = True)
metadata = []
i = 0
for conservative in [True]:
    for target_test_size in [.8, .4]:#np.linspace(0.2,0.8,4):
        for s in sigma_setting:
            for target_task in range(15):
                for decay_rate in [0.5, 1]:
                    for model_type in ["nn", "lm"]:
                        metadata.append({
                            "path": "exp" + str(i),
                            "n_tasks": 15,
                            "conservative": conservative,
                            "target_test_size": target_test_size,
                            "model_type": model_type,
                            "s": s,
                            "target_task": target_task,
                            "decay_rate": decay_rate
                        })
                        i += 1

metadf = pd.concat([pd.DataFrame(m, index=[i]) for i, m in enumerate(metadata)])
metadf.to_csv( base_dir / "metadata.csv")

sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}



In [5]:
for i, args in enumerate(metadata):
    working_path = Path(base_dir / args["path"])
    working_path.mkdir(parents = True, exist_ok = True)
    
    if args["model_type"] == "lm":
        loss_fn = mse
    elif args["model_type"] == "nn":
        loss_fn =  torch.nn.MSELoss()
    
    
    # generate data ------------------------------------------------
    np.random.seed(1234)
    f, betas, zs = random_functions(args["n_tasks"], 6,
                                    sigma_between = sigma_setting[args["s"]][0],
                                    sigma_within = sigma_setting[args["s"]][-1])
    result = []
    for i, fi in enumerate(f):
        x = np.random.uniform(0, 1, 100)
        result.append({
            "task": i,
            "x": x,
            "f": fi(x),
            "y": fi(x) + np.random.normal(0, .1, len(x))
        })
    # save data
    data_df = pd.concat([pd.DataFrame(r) for r in result])
    data_df.to_csv(working_path.joinpath("tasks.csv"), index = False)
    data_df = data_df.reset_index()
    betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
    betas_df = pd.DataFrame(betas_df)
    betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
    betas_df.to_csv(working_path / "betas.csv", index = False)
    data_dict = data_df.to_dict(orient = "list")
    
    input_data = prepare_input(data_dict,
                               target_task = args["target_task"],
                                target_test_size = args["target_test_size"],
                                preprocess = True)
    pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                               index = False)
        # bandit selection
    losses, alpha, beta, bandit_selects, pi, bl = bandit_source_train(input_data = input_data,
                                                                          model_class = args["model_type"],
                                                                          batch_size = 8,
                                                                          decay_rate = args["decay_rate"],
                                                                          n_it = 100,
                                                                          loss_fn =  loss_fn,
                                                                          conservative = args["conservative"],
                                                                     save_path = working_path)
        # save outputs
    save_files(working_path, alpha, beta, losses, bandit_selects, pi, bl)
    print(args)
    print(str(bl["bandit_final"]), str(bl["target_train"]))

{'path': 'exp0', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5}
[0.5923923254013062] [1.7780427932739258]
{'path': 'exp1', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'high_bw', 'target_task': 0, 'decay_rate': 0.5}
[0.8022153090296203] [1.9791431670076982]
{'path': 'exp2', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'high_bw', 'target_task': 0, 'decay_rate': 1}
[0.6285167336463928] [1.7907991409301758]
{'path': 'exp3', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'high_bw', 'target_task': 0, 'decay_rate': 1}
[1.979144732289658] [1.9791431670076982]
{'path': 'exp4', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'high_bw', 'target_task': 1, 'decay_rate': 0.5}
[0.505571722984314] [0.5091704726219177]
{'path': 'exp5', 'n_tasks': 15, 'con

{'path': 'exp43', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'high_bw', 'target_task': 10, 'decay_rate': 1}
[0.9423342650797227] [0.9390441703981747]
{'path': 'exp44', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'high_bw', 'target_task': 11, 'decay_rate': 0.5}
[0.3292509615421295] [0.31543442606925964]
{'path': 'exp45', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'high_bw', 'target_task': 11, 'decay_rate': 0.5}
[0.36369410421823783] [0.364825123658559]
{'path': 'exp46', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'high_bw', 'target_task': 11, 'decay_rate': 1}
[0.2684541642665863] [0.4404667317867279]
{'path': 'exp47', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'high_bw', 'target_task': 11, 'decay_rate': 1}
[0.3648252367971575] [0.364825123658559]
{'path': 'exp48', 'n_tasks

{'path': 'exp85', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 6, 'decay_rate': 0.5}
[0.07632752842844456] [0.07652240218271905]
{'path': 'exp86', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 6, 'decay_rate': 1}
[0.1473315954208374] [0.16257895529270172]
{'path': 'exp87', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 6, 'decay_rate': 1}
[0.07657326831461794] [0.07652240218271905]
{'path': 'exp88', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 7, 'decay_rate': 0.5}
[0.3017246425151825] [0.3478151261806488]
{'path': 'exp89', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 7, 'decay_rate': 0.5}
[0.25105261407508495] [0.25396893138428145]
{'path': 'ex

{'path': 'exp127', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'low_bw', 'target_task': 1, 'decay_rate': 1}
[0.17327018426577825] [0.17009521086919221]
{'path': 'exp128', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5}
[0.43136245012283325] [0.49603909254074097]
{'path': 'exp129', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'low_bw', 'target_task': 2, 'decay_rate': 0.5}
[0.43719639655453857] [0.4452297444108665]
{'path': 'exp130', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'low_bw', 'target_task': 2, 'decay_rate': 1}
[0.37727388739585876] [0.513234555721283]
{'path': 'exp131', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'low_bw', 'target_task': 2, 'decay_rate': 1}
[0.44550108367974506] [0.4452297444108665]
{'path': 'exp132', 'n_tas

{'path': 'exp170', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'low_bw', 'target_task': 12, 'decay_rate': 1}
[0.5542421340942383] [0.4947170615196228]
{'path': 'exp171', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'low_bw', 'target_task': 12, 'decay_rate': 1}
[0.5069687076644425] [0.5069408306596244]
{'path': 'exp172', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'low_bw', 'target_task': 13, 'decay_rate': 0.5}
[0.6170296669006348] [0.6012353301048279]
{'path': 'exp173', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'lm', 's': 'low_bw', 'target_task': 13, 'decay_rate': 0.5}
[0.5877821104675311] [0.5697881244249003]
{'path': 'exp174', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.8, 'model_type': 'nn', 's': 'low_bw', 'target_task': 13, 'decay_rate': 1}
[0.6353750824928284] [0.594201385974884]
{'path': 'exp175', 'n_tasks

{'path': 'exp212', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'high_bw', 'target_task': 8, 'decay_rate': 0.5}
[0.1768723428249359] [0.15729406476020813]
{'path': 'exp213', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'high_bw', 'target_task': 8, 'decay_rate': 0.5}
[0.15325479629893357] [0.15331371299364355]
{'path': 'exp214', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'high_bw', 'target_task': 8, 'decay_rate': 1}
[0.23879306018352509] [0.1328541338443756]
{'path': 'exp215', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'high_bw', 'target_task': 8, 'decay_rate': 1}
[0.15344164555778225] [0.15331371299364355]
{'path': 'exp216', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'high_bw', 'target_task': 9, 'decay_rate': 0.5}
[0.21702413260936737] [0.23225435614585876]
{'path': 'exp217

{'path': 'exp254', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 3, 'decay_rate': 1}
[0.558968186378479] [0.4530763626098633]
{'path': 'exp255', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 3, 'decay_rate': 1}
[0.5531104853347746] [0.5527726758756232]
{'path': 'exp256', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 4, 'decay_rate': 0.5}
[0.5030739903450012] [0.4491245746612549]
{'path': 'exp257', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 4, 'decay_rate': 0.5}
[0.4345387943038047] [0.4377000333322349]
{'path': 'exp258', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 4, 'decay_rate': 1}
[0.4861620366573334] [0.4396280348300934]
{'path': 'exp259'

{'path': 'exp296', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 14, 'decay_rate': 0.5}
[0.3518204689025879] [0.3171871304512024]
{'path': 'exp297', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 14, 'decay_rate': 0.5}
[0.37046869775676644] [0.3560113308658337]
{'path': 'exp298', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'medium_bw', 'target_task': 14, 'decay_rate': 1}
[0.5791504383087158] [0.3363431990146637]
{'path': 'exp299', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'medium_bw', 'target_task': 14, 'decay_rate': 1}
[0.3568942543349273] [0.3560113308658337]
{'path': 'exp300', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'low_bw', 'target_task': 0, 'decay_rate': 0.5}
[1.343464732170105] [1.40988290309906]
{'path': 'exp30

{'path': 'exp338', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'low_bw', 'target_task': 9, 'decay_rate': 1}
[0.1546115130186081] [0.250132292509079]
{'path': 'exp339', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'low_bw', 'target_task': 9, 'decay_rate': 1}
[0.12474274599794581] [0.1247360302902462]
{'path': 'exp340', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'low_bw', 'target_task': 10, 'decay_rate': 0.5}
[0.8389273881912231] [0.8519471883773804]
{'path': 'exp341', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'lm', 's': 'low_bw', 'target_task': 10, 'decay_rate': 0.5}
[0.9368089089564158] [0.9980707298212967]
{'path': 'exp342', 'n_tasks': 15, 'conservative': True, 'target_test_size': 0.4, 'model_type': 'nn', 's': 'low_bw', 'target_task': 10, 'decay_rate': 1}
[0.9131087064743042] [0.8910369873046875]
{'path': 'exp343', 'n_tasks'